#House Prices - Advanced Regression Techniques (Kaggle)

Modelo desenvolvido para a competição do Kaggle. O desafio pede a criação de um modelo preditivo para alugueis de casa usando um dataset com mais de 50 colunas.

#Imports

In [1]:
!pip install imbalanced-learn

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.decomposition import PCA
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

#Lendo CSVs

In [3]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
test_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


##Legenda do dataset

###Legendas resumidas

* **SalePrice**: Preço de venda do imóvel em dólares. Esta é a variável alvo que você está tentando prever.
* **MSSubClass**: Classe de construção
* **MSZoning**: Classificação geral de zoneamento
* **LotFrontage**: Metros lineares de frente do terreno conectados à rua
* **LotArea**: Tamanho do lote em pés quadrados
* **Street**: Tipo de acesso à via
* **Alley**: Tipo de acesso por viela
* **LotShape**: Formato geral do terreno
* **LandContour**: Nivelamento do terreno
* **Utilities**: Tipo de serviços públicos disponíveis
* **LotConfig**: Configuração do lote
* **LandSlope**: Inclinação do terreno
* **Neighborhood**: Localização física dentro dos limites da cidade de Ames
* **Condition1**: Proximidade de estrada principal ou ferrovia
* **Condition2**: Proximidade de estrada principal ou ferrovia (caso exista uma segunda)
* **BldgType**: Tipo de edificação
* **HouseStyle**: Estilo da residência
* **OverallQual**: Qualidade geral dos materiais e acabamento
* **OverallCond**: Avaliação da condição geral
* **YearBuilt**: Ano da construção original
* **YearRemodAdd**: Ano da reforma
* **RoofStyle**: Tipo de telhado
* **RoofMatl**: Material do telhado
* **Exterior1st**: Revestimento externo da casa
* **Exterior2nd**: Revestimento externo da casa (caso haja mais de um material)
* **MasVnrType**: Tipo de revestimento em alvenaria
* **MasVnrArea**: Área do revestimento em alvenaria em pés quadrados
* **ExterQual**: Qualidade do material externo
* **ExterCond**: Condição atual do material externo
* **Foundation**: Tipo de fundação
* **BsmtQual**: Altura/qualidade do porão
* **BsmtCond**: Condição geral do porão
* **BsmtExposure**: Tipo de exposição do porão (acesso externo ou nível do jardim)
* **BsmtFinType1**: Qualidade da área acabada do porão (tipo 1)
* **BsmtFinSF1**: Área acabada do porão tipo 1 em pés quadrados
* **BsmtFinType2**: Qualidade da segunda área acabada do porão (se existir)
* **BsmtFinSF2**: Área acabada do porão tipo 2 em pés quadrados
* **BsmtUnfSF**: Área não acabada do porão em pés quadrados
* **TotalBsmtSF**: Área total do porão em pés quadrados
* **Heating**: Tipo de aquecimento
* **HeatingQC**: Qualidade e condição do aquecimento
* **CentralAir**: Ar-condicionado central
* **Electrical**: Sistema elétrico
* **1stFlrSF**: Área do primeiro andar em pés quadrados
* **2ndFlrSF**: Área do segundo andar em pés quadrados
* **LowQualFinSF**: Área acabada de baixa qualidade (todos os andares)
* **GrLivArea**: Área habitável acima do nível do solo em pés quadrados
* **BsmtFullBath**: Banheiros completos no porão
* **BsmtHalfBath**: Lavabos no porão
* **FullBath**: Banheiros completos acima do nível do solo
* **HalfBath**: Lavabos acima do nível do solo
* **Bedroom**: Número de quartos acima do nível do porão
* **Kitchen**: Número de cozinhas
* **KitchenQual**: Qualidade da cozinha
* **TotRmsAbvGrd**: Total de cômodos acima do nível do solo (não inclui banheiros)
* **Functional**: Avaliação da funcionalidade da casa
* **Fireplaces**: Número de lareiras
* **FireplaceQu**: Qualidade da lareira
* **GarageType**: Localização do garage
* **GarageYrBlt**: Ano de construção do garage
* **GarageFinish**: Acabamento interno do garage
* **GarageCars**: Capacidade do garage em número de carros
* **GarageArea**: Área do garage em pés quadrados
* **GarageQual**: Qualidade do garage
* **GarageCond**: Condição do garage
* **PavedDrive**: Entrada pavimentada
* **WoodDeckSF**: Área do deck de madeira em pés quadrados
* **OpenPorchSF**: Área da varanda aberta em pés quadrados
* **EnclosedPorch**: Área da varanda fechada em pés quadrados
* **3SsnPorch**: Área da varanda para três estações em pés quadrados
* **ScreenPorch**: Área da varanda com tela em pés quadrados
* **PoolArea**: Área da piscina em pés quadrados
* **PoolQC**: Qualidade da piscina
* **Fence**: Qualidade da cerca
* **MiscFeature**: Característica adicional não coberta em outras categorias
* **MiscVal**: Valor em dólares da característica adicional
* **MoSold**: Mês da venda
* **YrSold**: Ano da venda
* **SaleType**: Tipo de venda
* **SaleCondition**: Condição da venda


###Legendas detalhadas

* **MSSubClass**: Identifica o tipo de residência envolvida na venda.
  - 20: Casa térrea (1 andar), construída a partir de 1946
  - 30: Casa térrea construída até 1945
  - 40: Casa térrea com sótão acabado
  - 45: Casa de 1½ andar com sótão não acabado
  - 50: Casa de 1½ andar com sótão acabado
  - 60: Casa de 2 andares construída a partir de 1946
  - 70: Casa de 2 andares construída até 1945
  - 75: Casa de 2½ andares
  - 80: Casa em níveis divididos (split/multi-level)
  - 85: Split foyer
  - 90: Duplex
  - 120: Casa térrea em PUD (Planned Unit Development)
  - 150: Casa de 1½ andar em PUD
  - 160: Casa de 2 andares em PUD
  - 180: PUD multinível
  - 190: Conversão para duas famílias

* **MSZoning**: Classificação geral de zoneamento.
  - A: Agrícola
  - C: Comercial
  - FV: Residencial Vila Flutuante
  - I: Industrial
  - RH: Residencial de alta densidade
  - RL: Residencial de baixa densidade
  - RP: Residencial parque de baixa densidade
  - RM: Residencial de média densidade

* **LotFrontage**: Comprimento linear da frente do terreno em contato com a rua.

* **LotArea**: Área total do lote em pés quadrados.

* **Street**: Tipo de acesso viário ao imóvel.
  - Grvl: Cascalho
  - Pave: Pavimentado

* **Alley**: Tipo de acesso por viela.
  - Grvl: Cascalho
  - Pave: Pavimentado
  - NA: Sem acesso por viela

* **LotShape**: Formato geral do terreno.
  - Reg: Regular
  - IR1: Levemente irregular
  - IR2: Moderadamente irregular
  - IR3: Irregular

* **LandContour**: Nivelamento do terreno.
  - Lvl: Plano
  - Bnk: Aclive acentuado
  - HLS: Encosta
  - Low: Depressão

* **Utilities**: Tipo de serviços públicos disponíveis.
  - AllPub: Todos os serviços públicos
  - NoSewr: Eletricidade, gás e água (fossa séptica)
  - NoSeWa: Apenas eletricidade e gás
  - ELO: Apenas eletricidade

* **LotConfig**: Configuração do lote.
  - Inside: Lote interno
  - Corner: Lote de esquina
  - CulDSac: Cul-de-sac
  - FR2: Frente para duas ruas
  - FR3: Frente para três ruas

* **LandSlope**: Inclinação do terreno.
  - Gtl: Suave
  - Mod: Moderada
  - Sev: Acentuada

* **Neighborhood**: Localização do imóvel dentro da cidade de Ames.
  - Blmngtn: Bloomington Heights
  - Blueste: Bluestem
  - BrDale: Briardale
  - BrkSide: Brookside
  - ClearCr: Clear Creek
  - CollgCr: College Creek
  - Crawfor: Crawford
  - Edwards: Edwards
  - Gilbert: Gilbert
  - IDOTRR: Iowa DOT e ferrovia
  - MeadowV: Meadow Village
  - Mitchel: Mitchell
  - Names: North Ames
  - NoRidge: Northridge
  - NPkVill: Northpark Villa
  - NridgHt: Northridge Heights
  - NWAmes: Northwest Ames
  - OldTown: Old Town
  - SWISU: Sul/Oeste da Iowa State University
  - Sawyer: Sawyer
  - SawyerW: Sawyer West
  - Somerst: Somerset
  - StoneBr: Stone Brook
  - Timber: Timberland
  - Veenker: Veenker

* **Condition1**: Proximidade do imóvel a condições externas.
* **Condition2**: Proximidade adicional a condições externas (se aplicável).
  - Artery: Via arterial
  - Feedr: Via coletora
  - Norm: Normal
  - RRNn / RRNe: Próximo à ferrovia
  - RRAn / RRAe: Adjacente à ferrovia
  - PosN: Próximo a área positiva (parque, área verde)
  - PosA: Adjacente a área positiva

* **BldgType**: Tipo de construção.
  - 1Fam: Casa unifamiliar
  - 2FmCon: Conversão para duas famílias
  - Duplx: Duplex
  - TwnhsE: Townhouse de canto
  - TwnhsI: Townhouse interna

* **HouseStyle**: Estilo da residência.
  - 1Story: Um andar
  - 1.5Fin / 1.5Unf: Um andar e meio (acabado/não acabado)
  - 2Story: Dois andares
  - 2.5Fin / 2.5Unf: Dois andares e meio
  - SFoyer: Split foyer
  - SLvl: Split level

* **OverallQual**: Avaliação geral da qualidade dos materiais e acabamento (1 = muito ruim, 10 = excelente).

* **OverallCond**: Avaliação geral da condição do imóvel (1 = muito ruim, 10 = excelente).

* **YearBuilt**: Ano de construção original.

* **YearRemodAdd**: Ano da reforma ou ampliação.

* **RoofStyle**: Tipo de telhado.
  - Flat, Gable, Gambrel, Hip, Mansard, Shed

* **RoofMatl**: Material do telhado.
  - ClyTile, CompShg, Membran, Metal, Roll, Tar&Grv, WdShake, WdShngl

* **Exterior1st / Exterior2nd**: Revestimento externo do imóvel (primário/secundário).

* **MasVnrType**: Tipo de revestimento em alvenaria.
  - BrkCmn, BrkFace, CBlock, Stone, None

* **MasVnrArea**: Área do revestimento em alvenaria (pés quadrados).

* **ExterQual / ExterCond**: Qualidade e condição do material externo.
  - Ex, Gd, TA, Fa, Po

* **Foundation**: Tipo de fundação.
  - BrkTil, CBlock, PConc, Slab, Stone, Wood
  
* **BsmtQual**: Avaliação da altura/qualidade do porão.
  - Ex: Excelente (100+ polegadas)
  - Gd: Boa (90–99 polegadas)
  - TA: Típica (80–89 polegadas)
  - Fa: Regular (70–79 polegadas)
  - Po: Ruim (< 70 polegadas)
  - NA: Sem porão

* **BsmtCond**: Avaliação da condição geral do porão.
  - Ex: Excelente
  - Gd: Boa
  - TA: Típica (leve umidade permitida)
  - Fa: Regular (umidade, rachaduras ou assentamento)
  - Po: Ruim (rachaduras severas, assentamento ou muita umidade)
  - NA: Sem porão

* **BsmtExposure**: Tipo de exposição do porão (acesso externo ou nível do jardim).
  - Gd: Boa exposição
  - Av: Exposição média
  - Mn: Exposição mínima
  - No: Sem exposição
  - NA: Sem porão

* **BsmtFinType1**: Avaliação da área acabada principal do porão.
  - GLQ: Boa área habitável
  - ALQ: Área habitável média
  - BLQ: Área habitável abaixo da média
  - Rec: Sala de recreação
  - LwQ: Baixa qualidade
  - Unf: Não acabado
  - NA: Sem porão

* **BsmtFinSF1**: Área acabada do porão (tipo 1) em pés quadrados.

* **BsmtFinType2**: Avaliação da segunda área acabada do porão (se existir).
  - GLQ: Boa área habitável
  - ALQ: Área habitável média
  - BLQ: Área habitável abaixo da média
  - Rec: Sala de recreação
  - LwQ: Baixa qualidade
  - Unf: Não acabado
  - NA: Sem porão

* **BsmtFinSF2**: Área acabada do porão (tipo 2) em pés quadrados.

* **BsmtUnfSF**: Área não acabada do porão em pés quadrados.

* **TotalBsmtSF**: Área total do porão em pés quadrados.

* **Heating**: Tipo de aquecimento.
  - Floor: Aquecimento pelo piso
  - GasA: Ar forçado a gás
  - GasW: Água quente ou vapor a gás
  - Grav: Fornalha por gravidade
  - OthW: Aquecimento por água/vapor não a gás
  - Wall: Aquecedor de parede

* **HeatingQC**: Qualidade e condição do aquecimento.
  - Ex: Excelente
  - Gd: Boa
  - TA: Típica/Média
  - Fa: Regular
  - Po: Ruim

* **CentralAir**: Ar-condicionado central.
  - Y: Sim
  - N: Não

* **Electrical**: Sistema elétrico.
  - SBrkr: Disjuntores padrão e fiação Romex
  - FuseA: Caixa de fusíveis > 60 AMP (médio)
  - FuseF: Caixa de fusíveis 60 AMP (regular)
  - FuseP: Fiação antiga (ruim)
  - Mix: Sistema misto

* **1stFlrSF**: Área do primeiro andar em pés quadrados.

* **2ndFlrSF**: Área do segundo andar em pés quadrados.

* **LowQualFinSF**: Área acabada de baixa qualidade (todos os andares).

* **GrLivArea**: Área habitável acima do nível do solo em pés quadrados.

* **BsmtFullBath**: Banheiros completos no porão.

* **BsmtHalfBath**: Lavabos no porão.

* **FullBath**: Banheiros completos acima do nível do solo.

* **HalfBath**: Lavabos acima do nível do solo.

* **Bedroom**: Número de quartos acima do nível do solo (não inclui porão).

* **Kitchen**: Número de cozinhas acima do nível do solo.

* **KitchenQual**: Qualidade da cozinha.
  - Ex: Excelente
  - Gd: Boa
  - TA: Típica/Média
  - Fa: Regular
  - Po: Ruim

* **TotRmsAbvGrd**: Total de cômodos acima do nível do solo (não inclui banheiros).

* **Functional**: Funcionalidade da residência.
  - Typ: Funcionalidade típica
  - Min1: Pequenas deduções
  - Min2: Pequenas deduções adicionais
  - Mod: Deduções moderadas
  - Maj1: Deduções severas
  - Maj2: Deduções muito severas
  - Sev: Severamente danificado
  - Sal: Apenas para salvamento

* **Fireplaces**: Número de lareiras.

* **FireplaceQu**: Qualidade da lareira.
  - Ex: Excelente (alvenaria de alto padrão)
  - Gd: Boa
  - TA: Média
  - Fa: Regular
  - Po: Ruim
  - NA: Sem lareira

* **GarageType**: Tipo/localização da garagem.
  - 2Types: Mais de um tipo
  - Attchd: Anexa à casa
  - Basment: Garagem no porão
  - BuiltIn: Integrada à casa
  - CarPort: Carport
  - Detchd: Separada da casa
  - NA: Sem garagem

* **GarageYrBlt**: Ano de construção da garagem.

* **GarageFinish**: Acabamento interno da garagem.
  - Fin: Acabada
  - RFn: Parcialmente acabada
  - Unf: Não acabada
  - NA: Sem garagem

* **GarageCars**: Capacidade da garagem em número de carros.

* **GarageArea**: Área da garagem em pés quadrados.

* **GarageQual**: Qualidade da garagem.
  - Ex, Gd, TA, Fa, Po, NA

* **GarageCond**: Condição da garagem.
  - Ex, Gd, TA, Fa, Po, NA

* **PavedDrive**: Tipo de acesso da garagem.
  - Y: Pavimentado
  - P: Parcialmente pavimentado
  - N: Terra/cascalho

* **WoodDeckSF**: Área do deck de madeira em pés quadrados.

* **OpenPorchSF**: Área da varanda aberta em pés quadrados.

* **EnclosedPorch**: Área da varanda fechada em pés quadrados.

* **3SsnPorch**: Área da varanda de três estações em pés quadrados.

* **ScreenPorch**: Área da varanda com tela em pés quadrados.

* **PoolArea**: Área da piscina em pés quadrados.

* **PoolQC**: Qualidade da piscina.
  - Ex: Excelente
  - Gd: Boa
  - TA: Média
  - Fa: Regular
  - NA: Sem piscina

* **Fence**: Qualidade da cerca.
  - GdPrv: Boa privacidade
  - MnPrv: Privacidade mínima
  - GdWo: Boa cerca de madeira
  - MnWw: Madeira/arame mínimo
  - NA: Sem cerca

* **MiscFeature**: Característica adicional não classificada.
  - Elev: Elevador
  - Gar2: Segunda garagem
  - Othr: Outro
  - Shed: Galpão
  - TenC: Quadra de tênis
  - NA: Nenhuma

* **MiscVal**: Valor em dólares da característica adicional.

* **MoSold**: Mês da venda (MM).

* **YrSold**: Ano da venda (YYYY).

* **SaleType**: Tipo de venda.
  - WD: Escritura convencional
  - CWD: Escritura à vista
  - VWD: Escritura com financiamento VA
  - New: Imóvel novo
  - COD: Venda judicial
  - Con / ConLw / ConLI / ConLD: Contratos especiais
  - Oth: Outro

* **SaleCondition**: Condição da venda.
  - Normal: Venda normal
  - Abnorml: Venda anormal (leilão, foreclosure)
  - AdjLand: Compra de terreno adjacente
  - Alloca: Alocação entre propriedades
  - Family: Venda entre familiares
  - Partial: Imóvel não concluído


#Análise dos dados

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [5]:
#Selecionando colunas com valores nulos substituíveis
columns_with_null = ['Alley', 'MasVnrType', 'FireplaceQu', 'PoolQC', 'Fence',
                     'MiscFeature', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
                     'BsmtFinType1', 'BsmtFinType2']

#Substituindo os valores nulos pela string 'NA'
train_df[columns_with_null] = train_df[columns_with_null].fillna('NA')
test_df[columns_with_null] = test_df[columns_with_null].fillna('NA')

#Removendo os valores nulos restantes
object_cols = train_df.select_dtypes(include=['object']).columns.tolist()
train_df[object_cols] = train_df[object_cols].dropna()
test_df[object_cols] = test_df[object_cols].dropna()

train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1378 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1378 non-null   object 
 6   Alley          1378 non-null   object 
 7   LotShape       1378 non-null   object 
 8   LandContour    1378 non-null   object 
 9   Utilities      1378 non-null   object 
 10  LotConfig      1378 non-null   object 
 11  LandSlope      1378 non-null   object 
 12  Neighborhood   1378 non-null   object 
 13  Condition1     1378 non-null   object 
 14  Condition2     1378 non-null   object 
 15  BldgType       1378 non-null   object 
 16  HouseStyle     1378 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

#Divisão treino e teste

In [6]:
X = train_df.drop('SalePrice', axis=1)
y = train_df['SalePrice']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f'X shape: {X_train.shape}')
print(f'y shape: {y_train.shape}')

X shape: (1168, 80)
y shape: (1168,)


#Construção da pipeline

In [8]:
object_cols = X_train.select_dtypes(include='object').columns

pre = ColumnTransformer([
  ('drop_cols', 'drop', ['Id']),
  ('num', SimpleImputer(strategy='median'), ['LotFrontage', 'MasVnrArea']),
  ('cat', OneHotEncoder(handle_unknown='ignore'), object_cols)
])

In [9]:
pipe = Pipeline([
  ('prep', pre),
  ('scaler', StandardScaler(with_mean=False)),
  ('clf', RandomForestRegressor())
])

In [10]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('drop_cols', 'drop', ['Id']),
                                                 ('num',
                                                  SimpleImputer(strategy='median'),
                                                  ['LotFrontage',
                                                   'MasVnrArea']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'H...
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object'))])),
                ('scaler', StandardScaler(with_mean=False)),
                ('clf', RandomForestRegressor())])

In [11]:
y_pred = pipe.predict(X_test)

In [12]:
X_processed = pipe.named_steps['prep'].transform(X_train)
X_processed.shape

(1168, 302)

In [13]:
'''pca = pipe.named_steps['pca']
explained_variance = pca.explained_variance_ratio_
cumulative_variance = pca.explained_variance_ratio_.cumsum()
'''

"pca = pipe.named_steps['pca']\nexplained_variance = pca.explained_variance_ratio_\ncumulative_variance = pca.explained_variance_ratio_.cumsum()\n"

In [14]:
'''
plt.plot(range(1, 126), cumulative_variance, marker='o', linestyle='--')
plt.title('Variância Explicada Acumulada por Componente')
plt.xlabel('Número de Componentes')
plt.ylabel('Variância Explicada Acumulada')
plt.grid(True)
plt.show()
'''

"\nplt.plot(range(1, 126), cumulative_variance, marker='o', linestyle='--')\nplt.title('Variância Explicada Acumulada por Componente')\nplt.xlabel('Número de Componentes')\nplt.ylabel('Variância Explicada Acumulada')\nplt.grid(True)\nplt.show()\n"

#Treinamento

In [15]:
param_grid = {
    'clf__n_estimators': [200, 300, 500],
    'clf__max_depth': [None, 10, 20, 30],
    'clf__min_samples_split': [2, 5, 10],
    'clf__min_samples_leaf': [1, 2, 4]
}

grid = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    scoring='r2',
    cv=5,
    n_jobs=-1,
    error_score='raise'
)

grid.fit(X_train, y_train)

y_pred = grid.predict(X_test)


#Avaliação

In [16]:
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.4f}")

MAE: 24213.77
MSE: 1446678171.31
RMSE: 38035.22
R²: 0.8114


###Interpretação das métricas

*   MAE (Erro Absoluto Médio): É a média dos erros absolutos entre os valores previstos pelo modelo e os valores reais. è medido no mesmo valor da variável. Quanto menor o valor, melhor pois representa que o modelo está mais preciso.

*   MSE (Erro Quadrático Médio): Semelhante ao MAE, mas o calcula é elevado ao quadrado. É sensível a outliers mas costuma representar melhor a variação dos erros.

*   RMSE (Raiz do Erro Quadrático Médio): É a raiz quadrada do MSE. Penaliza erros grandes mas mantém simple a interpretação.

* R²: Mede o quanto o modelo consegue explicar a variabilidade dos dados reais. Varia entre 0 e 1 onde 1 representa a previsão perfeita (o modelo acertou todos os valores) e 0 que o modelo errou todas as previsões. Deve ser analisado em conjunto com o MAE, MSE e RMSE pois um valor 1 (perfeito) geralmente está relacionado a problemas como overfitting, por exemplo.



In [19]:
y_pred_kagle = grid.predict(test_df)
sub_rf = pd.Series(y_pred_kagle, index=test_df['Id'], name = 'SalePrice')

In [20]:
sub_rf.to_csv('rf.csv', header=True)